# Bedrock AgentCore Gateway를 사용하여 YouTube API를 MCP 도구로 변환하기

## 개요
고객은 JSON 또는 YAML 형식의 OpenAPI 스펙을 가져와서 Bedrock AgentCore Gateway를 사용하여 API를 MCP 도구로 변환할 수 있습니다. 이 튜토리얼에서는 API 키를 사용하여 YouTube Data API를 호출하는 YouTube 검색 에이전트를 구축하는 방법을 보여드리겠습니다.

Gateway 워크플로우는 에이전트를 외부 도구에 연결하기 위해 다음 단계를 포함합니다:
* **Gateway용 도구 생성** - REST API용 OpenAPI 사양과 같은 스키마를 사용하여 도구를 정의합니다. OpenAPI 사양은 Gateway 생성을 위해 Amazon Bedrock AgentCore에서 파싱됩니다.
* **Gateway 엔드포인트 생성** - 인바운드 인증과 함께 MCP 진입점 역할을 할 게이트웨이를 생성합니다.
* **Gateway에 타겟 추가** - 게이트웨이가 특정 도구로 요청을 라우팅하는 방법을 정의하는 OpenAPI 타겟을 구성합니다. OpenAPI 파일의 모든 API는 MCP 호환 도구가 되며, Gateway 엔드포인트 URL을 통해 사용할 수 있게 됩니다. 각 OpenAPI Gateway 타겟에 대해 아웃바운드 인증을 구성합니다.
* **에이전트 코드 업데이트** - 통합된 MCP 인터페이스를 통해 모든 구성된 도구에 액세스하기 위해 에이전트를 Gateway 엔드포인트에 연결합니다.

![작동 방식](./assets/openapi-gateway-apikey.png)

### 튜토리얼 세부사항


| 정보          | 세부사항                                                   |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형        | 인터랙티브                                               |
| AgentCore 구성요소 | AgentCore Gateway, AgentCore Identity                     |
| 에이전트 프레임워크    | Strands Agents                                            |
| Gateway 타겟 유형  | OpenAPI                                                   |
| 에이전트                | YouTube 검색 에이전트                                      |
| 인바운드 인증 IdP     | Amazon Cognito                                            |
| 아웃바운드 인증        | API Key                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| 튜토리얼 구성요소  | AgentCore Gateway 생성 및 AgentCore Gateway 호출 |
| 튜토리얼 분야    | 범용                                            |
| 예제 복잡도   | 쉬움                                                      |
| 사용된 SDK             | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 AmazonCore Gateway 타겟을 생성합니다.

### 튜토리얼 아키텍처
이 튜토리얼에서는 OpenAPI yaml/json 파일에 정의된 작업을 MCP 도구로 변환하고 Bedrock AgentCore Gateway에서 호스팅합니다.
데모 목적으로 YouTube Data API를 사용하여 비디오를 검색하는 YouTube 검색 에이전트를 구축합니다. 에이전트는 YouTube Data API v3을 사용합니다. 솔루션은 Amazon Bedrock 모델을 사용하는 Strands Agent를 사용합니다.
예제에서는 YouTube 비디오 검색을 위한 searchVideos 도구가 있는 매우 간단한 에이전트를 사용합니다.

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook (Python 커널)
* AWS 자격 증명
* Amazon Cognito

In [1]:
#!pip install --force-reinstall -U -r requirements.txt 

In [7]:
# AWS 자격 증명 설정 (SageMaker 및 로컬 환경 지원)
import os
import boto3

# SageMaker 환경 확인
is_sagemaker = os.path.exists('/opt/ml') or 'SM_' in os.environ

if not is_sagemaker:
    # 로컬 환경: .env 파일 로드
    try:
        from dotenv import load_dotenv
        load_dotenv()
        
        # 필수 환경 변수 확인
        required_vars = ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']
        for var in required_vars:
            if not os.getenv(var):
                raise ValueError(f"환경 변수 {var}가 설정되지 않았습니다.")
    except ImportError:
        print("dotenv 패키지가 없습니다. 환경 변수가 직접 설정되어 있는지 확인하세요.")

# AWS_DEFAULT_REGION 설정
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')

# AWS 자격 증명 확인
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials:
        env_type = "SageMaker" if is_sagemaker else "로컬"
        print(f"AWS 자격 증명이 성공적으로 로드되었습니다. ({env_type} 환경, Region: {os.environ['AWS_DEFAULT_REGION']})")
    else:
        raise ValueError("AWS 자격 증명을 찾을 수 없습니다.")
except Exception as e:
    print(f"AWS 자격 증명 확인 중 오류: {e}")

AWS 자격 증명이 성공적으로 로드되었습니다. (SageMaker 환경, Region: us-east-1)


In [8]:
# 게이트웨이가 사용할(assume) IAM 역할 생성
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '404ef478-1734-40c9-b58c-8cfb6003705c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 25 Oct 2025 15:15:13 GMT', 'x-amzn-requestid': '404ef478-1734-40c9-b58c-8cfb6003705c', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-sample-lambdagateway-role
recreating agentcore-sample-lambdagateway-role
attaching role policy agentcore-sample-lambdagateway-role
Agentcore gateway role ARN:  arn:aws:iam::603420654815:role/agentcore-sample-lambdagateway-role


# 게이트웨이로의 인바운드 인증을 위한 Amazon Cognito 풀 생성

In [9]:
# Cognito User Pool 생성
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "읽기 액세스"},
    {"ScopeName": "gateway:write", "ScopeDescription": "쓰기 액세스"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito 리소스 생성 또는 검색 중...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("리소스 서버 확인됨.")

client_id, client_secret = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Discovery URL 가져오기
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)


Cognito 리소스 생성 또는 검색 중...
Found domain for user pool us-east-1_M7dGzKUpX: us-east-1m7dgzkupx (https://us-east-1m7dgzkupx.auth.us-east-1.amazoncognito.com)
User Pool ID: us-east-1_M7dGzKUpX
리소스 서버 확인됨.
Client ID: to73v0r7hg2h6f21m7d0ld2lo
https://cognito-idp.us-east-1.amazonaws.com/us-east-1_M7dGzKUpX/.well-known/openid-configuration


# 게이트웨이 생성하기  

In [18]:
import boto3
from botocore.exceptions import ClientError

gateway_client = boto3.client('bedrock-agentcore-control', region_name='us-east-1')
gateway_name = 'DemoGWOpenAPIAPIKeyYouTube'
create_response = None

# Cognito 인증 설정
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],
        "discoveryUrl": cognito_discovery_url
    }
}

try:
    # 기존 게이트웨이 확인 (올바른 키 사용)
    list_response = gateway_client.list_gateways()
    gateways = list_response.get('items', [])  # ✅ 'items' 키 사용
    
    for gateway in gateways:
        if gateway['name'] == gateway_name:
            print(f"✅ 기존 게이트웨이 '{gateway_name}' 사용!")
            create_response = gateway
            break
    
    if not create_response:
        print(f"새 게이트웨이 '{gateway_name}' 생성...")
        
        # 새 게이트웨이 생성
        create_response = gateway_client.create_gateway(
            name=gateway_name,
            roleArn=agentcore_gateway_iam_role['Role']['Arn'],
            protocolType='MCP',
            authorizerType='CUSTOM_JWT',
            authorizerConfiguration=auth_config,
            description='AgentCore Gateway with OpenAPI target'
        )
        print(f"✅ 새 게이트웨이 '{gateway_name}' 생성 완료!")
        
except ClientError as e:
    if e.response['Error']['Code'] == 'ConflictException':
        print(f"⚠️ ConflictException 발생: {e}")
        print("기존 게이트웨이를 다시 조회합니다...")
        
        # ConflictException 발생 시 다시 조회
        try:
            list_response = gateway_client.list_gateways()
            gateways = list_response.get('items', [])
            
            for gateway in gateways:
                if gateway['name'] == gateway_name:
                    print(f"✅ 기존 게이트웨이 '{gateway_name}' 찾음!")
                    create_response = gateway
                    break
                    
            if not create_response:
                raise Exception(f"게이트웨이 '{gateway_name}'를 찾을 수 없습니다.")
                
        except Exception as retry_error:
            print(f"❌ 재조회 실패: {retry_error}")
            raise retry_error
    else:
        print(f"❌ 게이트웨이 생성 실패: {e}")
        raise e
        
except Exception as e:
    print(f"❌ 예상치 못한 오류: {e}")
    raise e

# 결과 처리
if create_response:
    gatewayID = create_response["gatewayId"]
    
    # gatewayURL 처리 (API 응답에 있으면 사용, 없으면 구성)
    if "gatewayUrl" in create_response:
        gatewayURL = create_response["gatewayUrl"]
    else:
        gatewayURL = f"https://{gatewayID}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
    
    print(f"\n🎉 게이트웨이 준비 완료!")
    print(f"Gateway ID: {gatewayID}")
    print(f"Gateway URL: {gatewayURL}")
    print(f"Gateway Name: {gateway_name}")
    print(f"Status: {create_response.get('status', 'Unknown')}")
    
else:
    raise Exception("게이트웨이 생성 또는 조회에 실패했습니다.")

✅ 기존 게이트웨이 'DemoGWOpenAPIAPIKeyYouTube' 사용!

🎉 게이트웨이 준비 완료!
Gateway ID: demogwopenapiapikeyyoutube-rq7bi6cizg
Gateway URL: https://demogwopenapiapikeyyoutube-rq7bi6cizg.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
Gateway Name: DemoGWOpenAPIAPIKeyYouTube
Status: READY


# Bedrock AgentCore Gateway를 활용한 YouTube 오픈 API의 MCP 도구 변환

YouTube 비디오 검색 데이터를 가져오는 YouTube 검색 에이전트를 만들 예정입니다. YouTube Data API v3를 사용하려면 Google Cloud Console에서 API 키를 등록해야 합니다. [여기](https://console.developers.google.com/)에서 등록할 수 있습니다. 무료입니다! 

## YouTube API 키 발급 단계:

1. **Google Cloud Console 접속**: [console.developers.google.com](https://console.developers.google.com/)에 접속
2. **새 프로젝트 생성** 또는 기존 프로젝트 선택
3. **API 및 서비스 > 라이브러리**로 이동
4. **YouTube Data API v3** 검색 후 활성화
5. **API 및 서비스 > 사용자 인증 정보**로 이동
6. **사용자 인증 정보 만들기 > API 키** 선택
7. 생성된 API 키를 복사

## .env 파일 설정

API 키를 발급받은 후, 프로젝트 루트 디렉토리에 `.env` 파일을 생성하고 다음과 같이 추가하세요:

```bash
# .env 파일
AWS_ACCESS_KEY_ID=your_aws_access_key_here
AWS_SECRET_ACCESS_KEY=your_aws_secret_key_here
AWS_DEFAULT_REGION=us-east-1
YOUTUBE_API_KEY=your_youtube_key_here


# 게이트웨이로의 아웃바운드 인증을 위한 API KEY 자격 증명 공급자 생성

In [19]:
import boto3
import os
import time
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()
youtube_api_key = os.getenv('YOUTUBE_API_KEY')

acps = boto3.client(service_name="bedrock-agentcore-control")

# 고유한 이름으로 생성
timestamp = int(time.time())
credential_provider_name = f"YouTubeAPIKey_{timestamp}"

try:
    print(f"새 이름으로 자격 증명 공급자 생성: {credential_provider_name}")
    response = acps.create_api_key_credential_provider(
        name=credential_provider_name,
        apiKey=youtube_api_key,
    )
    
    credentialProviderARN = response['credentialProviderArn']
    print("✅ 새 자격 증명 공급자 생성 완료!")
    print(f"ARN: {credentialProviderARN}")
    
except Exception as e:
    print(f"❌ 생성 실패: {e}")


새 이름으로 자격 증명 공급자 생성: YouTubeAPIKey_1761408060
✅ 새 자격 증명 공급자 생성 완료!
ARN: arn:aws:bedrock-agentcore:us-east-1:603420654815:token-vault/default/apikeycredentialprovider/YouTubeAPIKey_1761408060


# OpenAPI Target 생성

In [20]:
# S3 클라이언트 생성
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS 계정 ID 및 리전 검색
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

# 매개변수 정의
# OpenAPI json 파일을 업로드할 S3 버킷 지정하기
bucket_name = f'agentcore-gateway-{account_id}-{region}'
file_path = 'assets/youtube_api_openapi.json'
object_key = 'youtube_api_openapi.json'

# put_object를 사용하여 파일 업로드 및 응답 읽기
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # 계정 ID와 리전으로 업로드된 객체의 URI 구성
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'업로드된 객체 S3 URI: {openapi_s3_uri}')
    
except Exception as e:
    print(f'파일 업로드 오류: {e}')


업로드된 객체 S3 URI: s3://agentcore-gateway-603420654815-us-east-1/youtube_api_openapi.json


# 아웃바운드 인증 구성 및 게이트웨이 타겟 생성

In [21]:
from botocore.exceptions import ClientError
import time

# 설정
youtube_openapi_s3_target_config = {
    "mcp": {
        "openApiSchema": {
            "s3": {
                "uri": openapi_s3_uri
            }
        }
    }
}

api_key_credential_config = [
    {
        "credentialProviderType": "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                "credentialParameterName": "key", 
                "providerArn": credentialProviderARN,
                "credentialLocation": "QUERY_PARAMETER",
            }
        }
    }
]

# 올바른 이름 형식 사용 (하이픈만 사용)
timestamp = int(time.time())
targetname = f'DemoOpenAPITargetS3YouTube-{timestamp}'  # 언더스코어 → 하이픈

try:
    print(f"새 게이트웨이 타겟 생성: {targetname}")
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gatewayID,
        name=targetname,
        description='OpenAPI Target with S3Uri using SDK - Fresh',
        targetConfiguration=youtube_openapi_s3_target_config,
        credentialProviderConfigurations=api_key_credential_config
    )
    print(f"✅ 새 게이트웨이 타겟 '{targetname}' 생성 완료!")
    
except Exception as e:
    print(f"❌ 게이트웨이 타겟 생성 실패: {e}")
    raise e

print(f"게이트웨이 타겟 '{targetname}' 준비 완료!")


새 게이트웨이 타겟 생성: DemoOpenAPITargetS3YouTube-1761408065
✅ 새 게이트웨이 타겟 'DemoOpenAPITargetS3YouTube-1761408065' 생성 완료!
게이트웨이 타겟 'DemoOpenAPITargetS3YouTube-1761408065' 준비 완료!


# Strands Agent에서 Bedrock AgentCore Gateway 호출

Strands 에이전트는 Model Context Protocol (MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS 도구와 원활하게 통합됩니다. 이 통합을 통해 AI 에이전트와 AWS 서비스 간의 안전하고 표준화된 통신이 가능합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본적인 MCP API인 ListTools 및 InvokeTools를 노출하는 프로토콜 호환 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트나 SDK가 안전하고 표준화된 방식으로 사용 가능한 도구를 발견하고 상호 작용할 수 있습니다. Strands 에이전트가 AWS 서비스에 액세스해야 할 때, 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 [MCP Authorization 사양](https://modelcontextprotocol.org/specification/draft/basic/authorization)을 엄격히 준수하여 강력한 보안과 액세스 제어를 보장합니다. 이는 Strands 에이전트의 모든 도구 호출이 인증 단계를 거쳐 보안을 유지하면서 강력한 기능을 활성화한다는 것을 의미합니다.

예를 들어, Strands 에이전트가 MCP 도구에 액세스해야 할 때, 먼저 ListTools를 호출하여 사용 가능한 도구를 발견한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 모든 필요한 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트나 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI 에이전트 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.


# 인바운드 인증을 위해 Amazon Cognito에서 액세스 토큰 요청

In [22]:
print("Amazon Cognito 인증자에서 액세스 토큰 요청 중...도메인 이름 전파가 완료될 때까지 일정 시간 실패할 수 있습니다")
token_response = utils.get_token(user_pool_id, client_id, client_secret, scopeString, REGION)
token = token_response["access_token"]
print("토큰 응답:", token)


Amazon Cognito 인증자에서 액세스 토큰 요청 중...도메인 이름 전파가 완료될 때까지 일정 시간 실패할 수 있습니다
to73v0r7hg2h6f21m7d0ld2lo
82a4ojjt61spn282bipna82v3am22dc4frnhimsqfkfru9t855p
토큰 응답: eyJraWQiOiI4NDRcL2lxYXN3ZnFVbVJlaXZvSHk1cnFtZm1SOHR0VUxlZlR5M3VZMHJmdz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJ0bzczdjByN2hnMmg2ZjIxbTdkMGxkMmxvIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJzYW1wbGUtYWdlbnRjb3JlLWdhdGV3YXktaWRcL2dhdGV3YXk6d3JpdGUgc2FtcGxlLWFnZW50Y29yZS1nYXRld2F5LWlkXC9nYXRld2F5OnJlYWQiLCJhdXRoX3RpbWUiOjE3NjE0MDgwNzIsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX003ZEd6S1VwWCIsImV4cCI6MTc2MTQxMTY3MiwiaWF0IjoxNzYxNDA4MDcyLCJ2ZXJzaW9uIjoyLCJqdGkiOiJjNmU1MTkxMC00OWYwLTQ3ZTYtODgwZC00NjkyODMwYTIwNzEiLCJjbGllbnRfaWQiOiJ0bzczdjByN2hnMmg2ZjIxbTdkMGxkMmxvIn0.rA9Wn3zHOPjFSbcZhUVCqOA7rX3OIM-yhPdUsFvTaYC_MMlnjbU3B73y5K3nxxDUHkGF0K8wok9Y8WyFmvLL-2doJL_b2mOrH2085dXaKv8t8JVRyXlFpWCFri32fPPzB-kvj85DDhdTGJmIA13CNfshJXuIaUiAm_w0mPET_5YTTX1EC41c3AI2-7yVAcEL6yxHl8ivalmGBk7O7PTKs7C5-R1_Xp46PkVw7FRegh7e5vTEBbq

# Bedrock AgentCore Gateway를 사용하여 YouTube Open API를 호출하여 YouTube 에이전트에게 문의

In [25]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials에 구성된 IAM 그룹/사용자는 Bedrock 모델에 대한 접근 권한이 있어야 합니다

yourmodel = BedrockModel(
    #model_id="us.amazon.nova-pro-v1:0",
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
)

In [26]:
from strands import Agent
import logging
import os
from dotenv import load_dotenv
import json


# .env 파일에서 환경 변수 로드
load_dotenv()

# 루트 strands 로거 구성
logging.getLogger("strands").setLevel(logging.INFO)
logging.basicConfig(format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()])

# YouTube API 키 검증
youtube_api_key = os.getenv('YOUTUBE_API_KEY')
if not youtube_api_key:
    raise ValueError(
        "YOUTUBE_API_KEY가 .env 파일에 설정되지 않았습니다.\n"
        ".env 파일에 다음을 추가하세요:\n"
        "YOUTUBE_API_KEY=your_youtube_api_key_here"
    )

print(f"✅ YouTube API 키 로드됨: {youtube_api_key[:10]}...")

# 결과를 저장할 딕셔너리 생성
results = {}

with client:
  tools = client.list_tools_sync()
  agent = Agent(model=yourmodel, tools=tools)

  print(f"에이전트에 로드된 도구들: {agent.tool_names}")

  print("\n=== 도구 목록 확인 ===")
  tool_list_response = agent("안녕하세요, 사용 가능한 모든 도구를 나열해 주실 수 있나요?")
  # results['tool_list'] = tool_list_response  # 이 줄 제거

  print("\n=== 2025년 국내 신선식품 동향 ===")
  news_search_response = agent("2025년 국내 신선식품 동향을 확인할 수 있는 youtube 영상들을 찾아주세요. 한 2개 정도 찾아주세요")
  results['news_search'] = news_search_response  # 이것만 저장

print("\n🎉 모든 테스트 완료!")

✅ YouTube API 키 로드됨: AIzaSyCInR...


INFO | strands.telemetry.metrics | Creating Strands MetricsClient


에이전트에 로드된 도구들: ['DemoOpenAPITargetS3YouTube-1759246733___searchVideos', 'DemoOpenAPITargetS3YouTube-1759309315___searchVideos', 'DemoOpenAPITargetS3YouTube-1761408065___searchVideos']

=== 도구 목록 확인 ===
안녕하세요! 현재 사용 가능한 도구는 YouTube 비디오 검색 기능입니다. 이 도구는 YouTube에서 비디오를 검색하고 상세 정보를 가져올 수 있습니다.

사용 가능한 도구 목록:
1. YouTube 비디오 검색 (searchVideos) - YouTube에서 비디오를 검색하고 상세 정보를 제공합니다.

이 도구를 사용하려면 다음과 같은 필수 매개변수가 필요합니다:
- part: API 응답에 포함할 리소스 속성을 지정합니다.
- q: 검색할 쿼리 용어를 입력합니다.

선택적 매개변수로는:
- maxResults: 반환할 최대 항목 수
- order: 검색 결과 정렬 방식 (date, rating, relevance, title, videoCount, viewCount)
- type: 특정 유형의 리소스만 검색하도록 제한

도구를 사용하여 특정 주제에 대한 YouTube 비디오를 검색하시겠습니까?
=== 2025년 국내 신선식품 동향 ===
2025년 국내 신선식품 동향에 관한 YouTube 영상을 검색해 드리겠습니다. 검색어를 "2025년 국내 신선식품 동향"으로 설정하여 관련 영상을 찾아보겠습니다.
Tool #1: DemoOpenAPITargetS3YouTube-1761408065___searchVideos
검색 결과를 확인해보니, 2025년 국내 신선식품 동향에 대한 직접적인 영상보다는 2025년 식품 수출 부적합 사례에 관한 영상이 검색되었습니다. 좀 더 관련성 높은 영상을 찾기 위해 검색어를 수정하여 다시 검색해보겠습니다.
Tool #2: DemoOpenAPITargetS3YouTube-176

**문제: 아래 셀을 실행할 때 아래 오류가 발생하면 pydantic과 pydantic-core 버전 간 호환성 문제가 있음을 나타냅니다.**

```
TypeError: model_schema() got an unexpected keyword argument ‘generic_origin’
```
**해결 방법?**

pydantic==2.7.2와 pydantic-core 2.27.2가 호환되는지 확인해야 합니다. 완료 후 커널을 재시작하세요.

# Clear up
정리 과정의 일환으로 수동으로 삭제해야 할 수 있는 IAM 역할, IAM 정책, 자격 증명 공급자, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성됩니다. 이는 실행하는 예제에 따라 다릅니다.

## Delete the gateway (Optional)

In [58]:
# import utilsA
#utils.delete_gateway(gateway_client,gatewayID)